# Lab: Porting LSTNet to Amazon SageMaker

In this lab the LSTNet model is ported to be trained using Amazon Sagemaker.

In [1]:
import boto3
import os
import pandas as pd
import numpy as np
from sagemaker.mxnet import MXNet
from sagemaker import get_execution_role

In [2]:
bucket = 'eduthie-sagemaker-1'
prefix = 'lstnet'

role = get_execution_role()

data_dir = '../../data'
data_file_path = os.path.join(data_dir,'electricity.txt')

test_bucket_prefix = '/test/'
single_host_train_bucket_prefix = '/train/single_host/'

Load the electricity dataset from file and normalise between 0 and 1 by dividing each column by the max. There are 321 time series of electricity consumption with 26,304 data points in each. 

In [3]:
df = pd.read_csv(data_file_path,header=None)
max_columns = df.max().astype(np.float64)
df = df/max_columns # normalize
print(df.shape)

(26304, 321)


Split into training and test datasets and upload to s3.

In [ ]:
train_frac = 0.8

num_time_steps = len(df)
split_index = int(num_time_steps*train_frac)
train = df[0:split_index]
print('Training size {}'.format(len(train)))
test = df[split_index:]
print('Test size {}'.format(len(test)))

test_file_path = os.path.join(data_dir,'test.csv')
test.to_csv(test_file_path,header=None,index=False)
train_file_path = os.path.join(data_dir,'train.csv')
train.to_csv(train_file_path,header=None,index=False)

client = boto3.client('s3')
client.upload_file(test_file_path, bucket, prefix + test_bucket_prefix + 'test.csv')
client.upload_file(train_file_path, bucket, prefix + single_host_train_bucket_prefix + 'train.csv')

Test locally first to make sure there are no bugs in the code.

In [3]:
from lstnet_sagemaker import train
hyperparameters = {
    'conv_hid' : 10,
    'gru_hid' : 10,
    'skip_gru_hid' : 2,
    'skip' : 5,
    'ar_window' : 6,
    'window' : 24*7,
    'horizon' : 24,
    'learning_rate' : 0.01,
    'clip_gradient' : 10.,
    'batch_size' : 128,
    'epochs' : 1
}
channel_input_dirs = {'train':data_dir,'test':data_dir}
train(
    hyperparameters = hyperparameters,
    input_data_config = None,
    channel_input_dirs = channel_input_dirs,
    output_data_dir = os.path.join(data_dir,'output'),
    model_dir = None,
    num_gpus = 1,
    num_cpus = 1,
    hosts = ['alg-1'],
    current_host = 'alg-1')

Train file path ../../data/test.csv
Test file path ../../data/test.csv
Loading file ../../data/test.csv
Loading file ../../data/test.csv
Is it a file True
Data length 5261
Loading file ../../data/test.csv
Loading file ../../data/test.csv
Is it a file True
Data length 5261
Running on [gpu(0)]
Hosts ['alg-1']
Current Host alg-1
kvstore device
Training Start
Epoch 0: rmse 0.34164103617270786 time 4.5380 s
Final rmse 0.3448484685116758
Total training time: 8.391311883926392
Training End
None


Choose the hyperparameters

In [ ]:
hyperparameters = {
    'conv_hid' : 100,
    'gru_hid' : 100,
    'skip_gru_hid' : 5,
    'skip' : 24,
    'ar_window' : 24,
    'window' : 24*7,
    'horizon' : 24,
    'learning_rate' : 0.001,
    'clip_gradient' : 10.,
    'batch_size' : 64,
    'batch_size' : 512,
    'epochs' : 100
}

In [ ]:
lstnet1 = MXNet(entry_point='lstnet_sagemaker.py',
    source_dir='.',
    role=role,
    output_path='s3://{}/{}/output'.format(bucket, prefix),
    train_instance_count=1,
    train_instance_type='ml.p3.2xlarge',
    hyperparameters=hyperparameters)
lstnet1.fit(inputs={'train': 's3://{}/{}{}'.format(bucket, prefix, single_host_train_bucket_prefix),
    'test': 's3://{}/{}{}'.format(bucket, prefix, test_bucket_prefix)},wait=False)